In [1]:
import pandas as pd
from numpy import nan
import pandas_gbq
import matplotlib as mpl
import pydeck as pdk
import json
import datetime

/home/pedro/Modelos/Faculdade/hackaton-fgv/venv/lib/python3.11/site-packages/geopandas/_compat.py:153: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  set_use_pygeos()


In [21]:
query = """SELECT *
    FROM datario.adm_cor_comando.ocorrencias
    WHERE data_particao > DATE("2023-06-01")""" # Coloquei um limite pra não gastar 3GB do google quando for puxar
chamados_rjcor = pandas_gbq.read_gbq(query, project_id = "hackaton-fgv")
#chamados_rjcor.to_csv("rjcor.csv")
#chamados_rjcor = pd.read_csv("rjcor.csv")
chamados_rjcor

/home/pedro/Modelos/Faculdade/hackaton-fgv/venv/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2257: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,id_pop,id_evento,bairro,data_inicio,data_fim,prazo,descricao,gravidade,latitude,longitude,status,tipo,data_particao
0,1,105336,Barra da tijuca,2023-11-01 10:45:32+00:00,2023-11-01 11:37:06+00:00,Curto,Av. min. ivan lins - alt. ponte da joatinga - ...,Baixo,-23.007138,-43.305252,Fechado,Primario,2023-11-01
1,1,105359,Grajau,2023-11-01 17:32:55+00:00,2023-11-01 18:19:18+00:00,Curto,"Estr. grajaú/jpa - alt. km 4,5 - eng. novo - s...",Baixo,-22.923714,-43.258467,Fechado,Primario,2023-11-01
2,1,105316,Guadalupe,2023-11-01 06:56:06+00:00,2023-11-01 07:35:59+00:00,Curto,Av. brasil - alt. guadalupe - guadalupe - sent...,Baixo,-22.861892,-43.549415,Fechado,Primario,2023-11-01
3,1,105340,Água santa,2023-11-01 12:03:46+00:00,2023-11-01 13:05:55+00:00,Curto,Linha amarela - alt. pedágio - sent. fundão,Baixo,-22.908814,-43.314814,Fechado,Primario,2023-11-01
4,1,105406,Barra da tijuca,2023-11-02 19:54:37+00:00,2023-11-02 22:08:34+00:00,Curto,Av. das américas - alt. n° 4059 - barra da tij...,Baixo,-23.000598,-43.354677,Fechado,Primario,2023-11-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7385,1,102225,São cristóvão,2023-09-10 17:06:04+00:00,2023-09-10 17:25:41+00:00,Curto,Viaduto do gasômetro - alt. rodoviária - santo...,Medio,-22.898193,-43.211672,Fechado,Primario,2023-09-10
7386,1,101768,Praca da bandeira,2023-09-01 05:49:09+00:00,2023-09-01 06:11:50+00:00,Curto,Elevado paulo de frontin - rio comprido - sen...,Medio,-22.919729,-43.209590,Fechado,Primario,2023-09-01
7387,1,102040,Freguesia de jacarepaguá,2023-09-06 08:56:24+00:00,2023-09-06 09:13:36+00:00,Curto,Linha amarela - cidade de deus - sent. orla,Medio,-22.947957,-43.355575,Fechado,Primario,2023-09-06
7388,1,102725,Freguesia de jacarepaguá,2023-09-19 10:29:30+00:00,2023-09-19 11:42:58+00:00,Curto,Túnel rebouças - 2° galeria - sent. lagoa,Medio,-22.949183,-43.350800,Fechado,Primario,2023-09-19


In [22]:
chamados_rjcor["data_inicio"] = pd.to_datetime(chamados_rjcor["data_inicio"])
chamados_rjcor["data_inicio_string"] = chamados_rjcor.apply(lambda linha: linha["data_inicio"].strftime("%d/%m/%Y, %H:%M:%S"), axis=1)
chamados_rjcor

,id_pop,id_evento,bairro,data_inicio,data_fim,prazo,descricao,gravidade,latitude,longitude,status,tipo,data_particao,data_inicio_string
0,1,105336,Barra da tijuca,2023-11-01 10:45:32+00:00,2023-11-01 11:37:06+00:00,Curto,Av. min. ivan lins - alt. ponte da joatinga - ...,Baixo,-23.007138,-43.305252,Fechado,Primario,2023-11-01,"01/11/2023, 10:45:32"
1,1,105359,Grajau,2023-11-01 17:32:55+00:00,2023-11-01 18:19:18+00:00,Curto,"Estr. grajaú/jpa - alt. km 4,5 - eng. novo - s...",Baixo,-22.923714,-43.258467,Fechado,Primario,2023-11-01,"01/11/2023, 17:32:55"
2,1,105316,Guadalupe,2023-11-01 06:56:06+00:00,2023-11-01 07:35:59+00:00,Curto,Av. brasil - alt. guadalupe - guadalupe - sent...,Baixo,-22.861892,-43.549415,Fechado,Primario,2023-11-01,"01/11/2023, 06:56:06"
3,1,105340,Água santa,2023-11-01 12:03:46+00:00,2023-11-01 13:05:55+00:00,Curto,Linha amarela - alt. pedágio - sent. fundão,Baixo,-22.908814,-43.314814,Fechado,Primario,2023-11-01,"01/11/2023, 12:03:46"
4,1,105406,Barra da tijuca,2023-11-02 19:54:37+00:00,2023-11-02 22:08:34+00:00,Curto,Av. das américas - alt. n° 4059 - barra da tij...,Baixo,-23.000598,-43.354677,Fechado,Primario,2023-11-02,"02/11/2023, 19:54:37"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7385,1,102225,São cristóvão,2023-09-10 17:06:04+00:00,2023-09-10 17:25:41+00:00,Curto,Viaduto do gasômetro - alt. rodoviária - santo...,Medio,-22.898193,-43.211672,Fechado,Primario,2023-09-10,"10/09/2023, 17:06:04"
7386,1,101768,Praca da bandeira,2023-09-01 05:49:09+00:00,2023-09-01 06:11:50+00:00,Curto,Elevado paulo de frontin - rio comprido - sen...,Medio,-22.919729,-43.209590,Fechado,Primario,2023-09-01,"01/09/2023, 05:49:09"
7387,1,102040,Freguesia de jacarepaguá,2023-09-06 08:56:24+00:00,2023-09-06 09:13:36+00:00,Curto,Linha amarela - cidade de deus - sent. orla,Medio,-22.947957,-43.355575,Fechado,Primario,2023-09-06,"06/09/2023, 08:56:24"
7388,1,102725,Freguesia de jacarepaguá,2023-09-19 10:29:30+00:00,2023-09-19 11:42:58+00:00,Curto,Túnel rebouças - 2° galeria - sent. lagoa,Medio,-22.949183,-43.350800,Fechado,Primario,2023-09-19,"19/09/2023, 10:29:30"


In [23]:
chamados_rjcor["gravidade"].value_counts(dropna = False)

gravidade
Baixo     3864
Normal    2770
Medio      402
Média      339
Alta         8
None         4
Alto         3
Name: count, dtype: int64

In [31]:
gravidades = {
    "Normal": [255, 255, 0],
    "Média": [255, 165, 0],
    "Alta": [255, 0, 0],
    None: [200, 200, 200],
    "nan": [200, 200, 200],
    "Baixo": [255, 255, 0],
    "Medio": [255, 165, 0],
    "Alto": [255, 0, 0]
}
def aplica_cores(linha):
    if linha["status"] == "Fechado":
        return [0, 255, 0]
    else:
        return gravidades[linha["gravidade"]]
chamados_rjcor["coloracao"] = chamados_rjcor.apply(aplica_cores, axis=1)
chamados_rjcor

,id_pop,id_evento,bairro,data_inicio,data_fim,prazo,descricao,gravidade,latitude,longitude,status,tipo,data_particao,data_inicio_string,coloracao,raio
0,1,105336,Barra da tijuca,2023-11-01 10:45:32+00:00,2023-11-01 11:37:06+00:00,Curto,Av. min. ivan lins - alt. ponte da joatinga - ...,Baixo,-23.007138,-43.305252,Fechado,Primario,2023-11-01,"01/11/2023, 10:45:32","[0, 255, 0]",50
1,1,105359,Grajau,2023-11-01 17:32:55+00:00,2023-11-01 18:19:18+00:00,Curto,"Estr. grajaú/jpa - alt. km 4,5 - eng. novo - s...",Baixo,-22.923714,-43.258467,Fechado,Primario,2023-11-01,"01/11/2023, 17:32:55","[0, 255, 0]",50
2,1,105316,Guadalupe,2023-11-01 06:56:06+00:00,2023-11-01 07:35:59+00:00,Curto,Av. brasil - alt. guadalupe - guadalupe - sent...,Baixo,-22.861892,-43.549415,Fechado,Primario,2023-11-01,"01/11/2023, 06:56:06","[0, 255, 0]",50
3,1,105340,Água santa,2023-11-01 12:03:46+00:00,2023-11-01 13:05:55+00:00,Curto,Linha amarela - alt. pedágio - sent. fundão,Baixo,-22.908814,-43.314814,Fechado,Primario,2023-11-01,"01/11/2023, 12:03:46","[0, 255, 0]",50
4,1,105406,Barra da tijuca,2023-11-02 19:54:37+00:00,2023-11-02 22:08:34+00:00,Curto,Av. das américas - alt. n° 4059 - barra da tij...,Baixo,-23.000598,-43.354677,Fechado,Primario,2023-11-02,"02/11/2023, 19:54:37","[0, 255, 0]",50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7385,1,102225,São cristóvão,2023-09-10 17:06:04+00:00,2023-09-10 17:25:41+00:00,Curto,Viaduto do gasômetro - alt. rodoviária - santo...,Medio,-22.898193,-43.211672,Fechado,Primario,2023-09-10,"10/09/2023, 17:06:04","[0, 255, 0]",50
7386,1,101768,Praca da bandeira,2023-09-01 05:49:09+00:00,2023-09-01 06:11:50+00:00,Curto,Elevado paulo de frontin - rio comprido - sen...,Medio,-22.919729,-43.209590,Fechado,Primario,2023-09-01,"01/09/2023, 05:49:09","[0, 255, 0]",50
7387,1,102040,Freguesia de jacarepaguá,2023-09-06 08:56:24+00:00,2023-09-06 09:13:36+00:00,Curto,Linha amarela - cidade de deus - sent. orla,Medio,-22.947957,-43.355575,Fechado,Primario,2023-09-06,"06/09/2023, 08:56:24","[0, 255, 0]",50
7388,1,102725,Freguesia de jacarepaguá,2023-09-19 10:29:30+00:00,2023-09-19 11:42:58+00:00,Curto,Túnel rebouças - 2° galeria - sent. lagoa,Medio,-22.949183,-43.350800,Fechado,Primario,2023-09-19,"19/09/2023, 10:29:30","[0, 255, 0]",50


In [36]:
camera_inicial = pdk.ViewState(
    latitude = -22.9035, 
    longitude = -43.4096, 
    zoom = 9.3, 
    max_zoom = 16, 
    pitch = 0, 
    bearing = 0
)

base = pdk.Layer(
    "PolygonLayer",
    stroked = False,
    get_polygon = "-",
    get_fill_color = [0, 0, 0, 20],
)

ocorrencias_acontecendo = pdk.Layer(
    "ScatterplotLayer",
    chamados_rjcor[["latitude", "longitude", "coloracao", "bairro", "data_inicio_string", 
        "gravidade", "descricao", "status"]][chamados_rjcor["status"] == "Aberto"],
    get_position = ["longitude", "latitude"],
    auto_highlight = True,
    get_radius = "200",
    get_fill_color = "coloracao",
    pickable = True
)

ocorrencias_fechadas = pdk.Layer(
    "ScatterplotLayer",
    chamados_rjcor[["latitude", "longitude", "coloracao", "bairro", "data_inicio_string", 
        "gravidade", "descricao", "status"]][chamados_rjcor["status"] == "Fechado"],
    get_position = ["longitude", "latitude"],
    auto_highlight = True,
    get_radius = "100",
    get_fill_color = "coloracao",
    pickable = True
)

r = pdk.Deck(
    layers = [base, ocorrencias_acontecendo], 
    initial_view_state = camera_inicial,
    tooltip = {"html": "<b>Bairro:</b> {bairro}<br><b>Data de inicio:</b> {data_inicio_string}"\
               "<br><b>Gravidade:</b> {gravidade}<br><b>Endereço:</b> {descricao}"\
               "<br><b>Situação:</b> {status}",
               "style": {
                  "backgroundColor": "steelblue",
                  "color": "white"
                        }
              },
    map_style = "dark_no_labels"
)

r.to_html("geojson_layer.html")